In [37]:
import os
import pathlib
import pandas as pd


from helpers.column import COLUMN_HEAD

In [38]:
def get_final_year():
    allowed_years = ['2020', '2021', '2022']
    msg = f'Please provide final year to analize: [allowed are: {allowed_years}] '

    get_year = input(msg)
    while get_year not in allowed_years:
        get_year = input(msg)
    
    print(f'Will analyze data for {get_year}')
    return get_year

In [39]:
cleaned_avg_population_file = pathlib.Path(f'{os.getcwd()}/../../../cleaned_data/Population_all_ages_2017-2019(Bulgaria).csv')
pop = pd.read_csv(cleaned_avg_population_file)
pop = pop.loc[:, [COLUMN_HEAD.LOCATION, COLUMN_HEAD.SEX, COLUMN_HEAD.AGE,  '2019']]
pop.rename(columns={'2019': COLUMN_HEAD.POPULATION}, inplace=True)

In [40]:
life_tables_file = pathlib.Path(f'{os.getcwd()}/../../../output_data/life_table_calculations/Bulgaria_Abidged_life_table.csv')
life_tables = pd.read_csv(life_tables_file)
life_tables = life_tables.loc[:, [COLUMN_HEAD.LOCATION, COLUMN_HEAD.SEX, COLUMN_HEAD.AGE, 'e(x)']]
life_tables.rename(columns={'e(x)': COLUMN_HEAD.LIFE_EXPECTANCY}, inplace=True)

In [41]:
standardized_pop_file = pathlib.Path(f'{os.getcwd()}/../../../output_data/life_table_calculations/Bulgarian Standardized population.csv')
standardized_pop = pd.read_csv(standardized_pop_file)

In [42]:
final_year = get_final_year()
mortality_file = pathlib.Path(f'{os.getcwd()}/../../../output_data/Excess_mortality/Predicted_excess_mortality_by_location_sex_age_year:{final_year}.csv')
mortality = pd.read_csv(mortality_file)

Will analyze data for 2022


In [43]:
pop = pop.groupby([COLUMN_HEAD.AGE, COLUMN_HEAD.SEX, COLUMN_HEAD.LOCATION], as_index=False)[COLUMN_HEAD.POPULATION].sum()

In [44]:
exc_mort = mortality.merge(life_tables, on=[COLUMN_HEAD.AGE, COLUMN_HEAD.SEX, COLUMN_HEAD.LOCATION])

In [45]:
exc_mort[COLUMN_HEAD.PYLL_MEAN] = exc_mort.apply(lambda x:
                                          x[COLUMN_HEAD.LIFE_EXPECTANCY] * x[COLUMN_HEAD.EXCESS_MORTALITY_MEAN]
                                          if x[COLUMN_HEAD.EXCESS_MORTALITY_MEAN] > 0
                                          else 0,
                                          axis=1).round(1)

exc_mort[COLUMN_HEAD.PYLL_FLUCTUATION] = exc_mort.apply(lambda x:
                                                 x[COLUMN_HEAD.LIFE_EXPECTANCY] * x[COLUMN_HEAD.CONFIDENCE_INTERVAL]
                                                 if x[COLUMN_HEAD.PYLL_MEAN] > 0
                                                 else 0,
                                                 axis=1).round(1)

In [46]:
exc_mort = exc_mort.loc[(exc_mort[COLUMN_HEAD.IS_SIGNIFICANT] == 'Significant Increase'), :]

In [47]:
exc_mort = exc_mort.merge(pop, on=[COLUMN_HEAD.AGE, COLUMN_HEAD.SEX, COLUMN_HEAD.LOCATION])

In [48]:
exc_mort = exc_mort.merge(standardized_pop, on=[COLUMN_HEAD.AGE, COLUMN_HEAD.SEX])

In [49]:
exc_mort[COLUMN_HEAD.PYLL_RATE] = exc_mort.apply(lambda x:
                                    (x[COLUMN_HEAD.PYLL_MEAN] / x[COLUMN_HEAD.POPULATION])
                                    *
                                    10 ** 5,
                                    axis=1).round(2)

exc_mort[COLUMN_HEAD.PYLL_RATE_FLUC] = exc_mort.apply(lambda x:
                                        (x[COLUMN_HEAD.PYLL_FLUCTUATION] / x[COLUMN_HEAD.POPULATION])
                                        *
                                        10 ** 5,
                                        axis=1).round(2)

In [50]:
exc_mort[COLUMN_HEAD.POPULATION_PER_100_000] = exc_mort.apply(lambda x:
                                                (x[COLUMN_HEAD.STANDARD_POPULATION] / 100_000),
                                                axis=1).round(3)

In [51]:
exc_mort[COLUMN_HEAD.ASYR] = exc_mort.apply(lambda x:
                                (x[COLUMN_HEAD.PYLL_RATE] * x[COLUMN_HEAD.POPULATION_PER_100_000]),
                                axis=1).round(3)

exc_mort[COLUMN_HEAD.ASYR_FLUC] = exc_mort.apply(lambda x:
                                    (x[COLUMN_HEAD.PYLL_RATE_FLUC] * x[COLUMN_HEAD.POPULATION_PER_100_000]),
                                    axis=1).round(3)

In [52]:
agg_params = {COLUMN_HEAD.ASYR: 'sum',
              COLUMN_HEAD.ASYR_FLUC: 'sum', }
exc_mort = exc_mort.groupby([COLUMN_HEAD.LOCATION, COLUMN_HEAD.SEX], as_index=False).agg(agg_params)

In [53]:
exc_mort.sort_values([COLUMN_HEAD.SEX, COLUMN_HEAD.ASYR], ascending=False, inplace=True)

In [ ]:
asyr_file = pathlib.Path(f'{os.getcwd()}/../../../output_data/life_table_calculations/ASYR_for_year:{final_year}.csv')
exc_mort.to_csv(asyr_file, index=False)